In [2]:
import numpy as np
import math
import pymeshlab
import potpourri3d as pp3d
import polyscope as ps
from barmesh.tribarmes import trianglebarmesh
from barmesh.basicgeo import P3

In [3]:
#ms = pymeshlab.MeshSet()
fname = 'geometry/simple_geom.stl'
#fname = 'geometry/spot_triangulated.stl'
#fname = 'geometry/twoWonkyTriangles.stl'
#ms.load_new_mesh(fname)
#V, F = pp3d.read_mesh(fname)
tbm = trianglebarmesh.TriangleBarMesh(fname)
#n = ms.current_mesh().face_normal_matrix()
V = tbm.GetVertices()
F = tbm.GetFaces()
n = tbm.GetNormals()

In [4]:
def pointOnEdge(i, l):
    pt1 = tbm.bars[i].GetNodeFore(True).p
    pt2 = tbm.bars[i].GetNodeFore(False).p
    vec12 = (pt2[0]-pt1[0],pt2[1]-pt1[1],pt2[2]-pt1[2])
    ptl = P3(pt1[0]+l*vec12[0],pt1[1]+l*vec12[1],pt1[2]+l*vec12[2])
    return ptl

def P3list2array(P3list):
    pts =[]
    for p in P3list:
        pts.append((p.x,p.y,p.z))
    return np.array(pts)


def find_rot(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """

    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix

def find_intersection(p1,vec1,p2,vec2, tol= 1e-6):
    flip = vec2.y ==0
    if flip:
        p1,vec1,p2,vec2 = p2,vec2,p1,vec1
    a = (p2.x+(vec2.x*p1.y-vec2.x*p2.y)/vec2.y-p1.x) / (vec1.x - vec2.x*vec1.y/vec2.y)
    b = (p1.y+ a*vec1.y-p2.y)/vec2.y
    p_int1 = P3(p1.x+ a*vec1.x,p1.y+ a*vec1.y,p1.z+ a*vec1.z)
    #print('tol ',abs((p1.z+ a*vec1.z)-(p2.z+ b*vec2.z)))
    assert abs((p1.z+ a*vec1.z)-(p2.z+ b*vec2.z)) < tol, "Lines do not intersect in 3D space"    
    if flip:
        a = b
    return p_int1,a

#function to rotate a P3 vector about an axis (of any length) and an angle in degrees
def rotAxisAngle(v,ax,a):
    ax = P3.ZNorm(ax)
    c = np.cos(np.radians(a))
    s = np.sin(np.radians(a))
    C = 1 - np.cos(np.radians(a))
    Q= np.array(((ax.x*ax.x*C+c,      ax.x*ax.y*C-ax.z*s, ax.x*ax.z*C+ax.y*s),
                 (ax.y*ax.x*C+ax.z*s, ax.y*ax.y*C+c,      ax.y*ax.z*C-ax.x*s),
                 (ax.z*ax.x*C-ax.y*s, ax.z*ax.y*C+ax.x*s, ax.z*ax.z*C+c)))
    v = np.dot(Q,np.array((v.x,v.y,v.z)))
    return P3(v[0],v[1],v[2])
    


In [5]:
# axis is a b
# incoming at d = Along(lam, a, b)
# incoming from c to d 
# incoming (d - c).(b - a) = |d-c||b-a|cos(theta) = |d-c|I
# I = |b-a|cos(theta) = (d - c).(b - a)/|d-c|

# triangle on right hand side is to e
# outgoing to x where x = Along(q, a, e) or x = Along(q, b, e)
# Solve (x - d).(b - a) = |x-d||b-a|cos(theta) = |x-d|I

# Set d = 0, v = b - a
# x.v = |x|I  where x = a + (e - a)*q = a + f*q
# x.x I^2 = (x.v)^2

# x.x I^2 = (a + f*q)^2 I^2 = q^2 f^2 I^2 + 2q a.f I^2 + a^2 I^2
#   =
# (x.v)^2 = (a.v + f.v*q)^2 = q^2 (f.v)^2 + 2q (a.v)(f.v) + (a.v)^2
# q^2 ((f.v)^2 - f^2 I^2) + 2q ((a.v)(f.v) - a.f I^2) + (a.v)^2 - a^2 I^2

def Square(X):
    return X*X
def TOL_ZERO(X):
    if not (abs(X) < 0.0001):
        print("TOL_ZERO fail", X)

def GeoCrossAxisE(a, Vae, Vab, Isq, Isgn):
    # Solve: Isq*x.Lensq() - Square(P3.Dot(x, Vab)) = 0   for x = a + Vae*q
    # 0 = Isq*(a^2 + 2q a.Vae + q^2 Vae^2) - (a.Vab + Vae.Vab q)^2
    #   = Isq*(a^2 + 2q adf + q^2 Vae^2) - (adv + fdv q)^2

    fdv = P3.Dot(Vae, Vab)
    adv = P3.Dot(a, Vab)
    adf = P3.Dot(a, Vae)
    qA = Square(fdv) - Vae.Lensq()*Isq
    qB2 = adv*fdv - adf*Isq
    qC = Square(adv) - a.Lensq()*Isq
    if abs(qA) <= abs(qB2)*1e-7:
        if qB2 == 0:
            return -1.0
        q = -qC/(2*qB2)
    else:
        qdq = Square(qB2) - qA*qC
        if qdq < 0.0:
            #print("qdq", qdq)
            return -1.0
        qs = math.sqrt(qdq) / qA
        qm = -qB2 / qA
        q = qm + qs*Isgn
    # q = qs +- qm,  x = a + Vae*q,  Dot(x, Vab) same sign as Dot(Vcd, Vab)
    if abs(q) < 100:
        TOL_ZERO(qA*Square(q) + qB2*2*q + qC)
    return q

#
# This is the basic function that crosses from one triangle to the next
#
def GeoCrossAxis(Ga, Gb, Gc, lam, Ge):
    Vab = Gb - Ga
    Gd = Ga + Vab*lam
    Vcd = Gd - Gc
    cdDab = P3.Dot(Vcd, Vab)
    Isq = Square(cdDab) / Vcd.Lensq()
    Isgn = -1 if cdDab < 0 else 1
    qVae = GeoCrossAxisE(Ga - Gd, Ge - Ga, Vab, Isq, Isgn)
    qVbe = GeoCrossAxisE(Gb - Gd, Ge - Gb, -Vab, Isq, -Isgn)
    bAEcrossing = (abs(qVae - 0.5) < abs(qVbe - 0.5))
    q = qVae if bAEcrossing else qVbe
    Gx = (Ga + (Ge - Ga)*q) if bAEcrossing else (Gb + (Ge - Gb)*q)
    Dx = Gx - Gd
    TOL_ZERO(Isq - Square(P3.Dot(Dx, Vab)/Dx.Len()))
    TOL_ZERO(P3.Dot(Vcd, Vab)/Vcd.Len() - P3.Dot(Dx, Vab)/Dx.Len())
    return bAEcrossing, q, Gx

In [6]:
# This is the iterative function that goes from bar to bar through the mesh
# c=from point, bar the crossing edge, lam the point on the bar, and bGoRight is the crossing direction
#
def GeoCrossBar(c, bar, lam, bGoRight, bPrintvals=False):
    bEnd = False
    Na, Nb = bar.nodeback, bar.nodefore
    if bGoRight:
        if bar.barforeright == None:
            bEnd = True
            Ne = trianglebarmesh.TriangleNode(P3(bar.nodefore.p.x+0.01,bar.nodefore.p.y+0.01,bar.nodefore.p.z+0.01),-1)
        else:
            Ne = bar.barforeright.GetNodeFore(bar.barforeright.nodeback == bar.nodefore)
    else:
        if bar.barbackleft == None:
            bEnd = True
            Ne = trianglebarmesh.TriangleNode(P3(bar.nodeback.p.x+0.01,bar.nodeback.p.y+0.01,bar.nodeback.p.z+0.01),-1)
        else:
            Ne = bar.barbackleft.GetNodeFore(bar.barbackleft.nodeback == bar.nodeback)
    d = Na.p + (Nb.p - Na.p)*lam
    if bPrintvals:
        print("\n", (Na.p, Nb.p, c, lam, Ne.p))
    bAEcrossing, q, Gx = GeoCrossAxis(Na.p, Nb.p, c, lam, Ne.p)
    if bGoRight:
        if bAEcrossing:
            if not bEnd:
                bar = bar.barforeright.GetForeRightBL(bar.barforeright.nodeback == Nb)
                lam = q if bar.nodeback == Na else 1-q
                bGoRight = (bar.nodeback == Na)
            else:
                bar = None
        else:
            if not bEnd:
                bar = bar.barforeright
                lam = q if bar.nodeback == Nb else 1-q
                bGoRight = not (bar.nodeback == Nb)
            else:
                bar = None
    else:
        if bAEcrossing:
            if not bEnd:
                bar = bar.barbackleft
                lam = q if bar.nodeback == Na else 1-q
                bGoRight = not (bar.nodeback == Na)
            else:
                bar = None
        else:
            if not bEnd:
                bar = bar.barbackleft.GetForeRightBL(bar.barbackleft.nodeback == Na)
                lam = q if bar.nodeback == Nb else 1-q
                bGoRight = (bar.nodeback == Nb)
            else:
                bar = None
    if not bEnd:
        c = bar.nodeback.p + (bar.nodefore.p - bar.nodeback.p)*lam
        TOL_ZERO((c - Gx).Len())
    else:
        c = None
    if bPrintvals:
        print("d,c", (d, c))
    
    return (d, bar, lam, bGoRight)



In [7]:
startFace = 319

x,y,z = 0,0,0
for nn in tbm.faces[startFace].nodes:
    x += tbm.nodes[nn].p.x/3
    y += tbm.nodes[nn].p.y/3
    z += tbm.nodes[nn].p.z/3
    
startPt = P3(x,y,z)#
ref90 = P3(1,0,0)# A reference direction approximately perpendicular to the axis of the part
ref0 = P3.Cross(ref90,tbm.faces[startFace].normal)
thetas = np.arange(0,180,10)

barsints =[]
for theta in thetas:
    startVec = rotAxisAngle(ref0,tbm.faces[startFace].normal,theta)
    barsint =[]
    #find which bars intersect with the start vector
    for b in tbm.faces[startFace].bars:
        p1 = tbm.bars[b].GetNodeFore(True).p
        vec1 = tbm.bars[b].GetNodeFore(False).p-tbm.bars[b].GetNodeFore(True).p
        try:
            Pint , lam = find_intersection(p1,vec1,startPt,startVec,tol=1e-4)
        except:
            lam = -1
            print('Error finding instersection between starting lines and edge of face, try specifying ref90 in different plane')
        if 0 < lam < 1:
            barsint.append((b,1-lam))
    barsints.append(barsint)
        
geopaths=[]
curvs = []
faces =[]
barF,bGoRightF,lamF,barB,bGoRightB,lamB = [],[],[],[],[],[]



for barsint in barsints:
    barF.append(tbm.bars[barsint[0][0]])
    bGoRightF.append(barF[-1].faceleft == startFace)
    lamF.append(barsint[0][1])
    barB.append(tbm.bars[barsint[1][0]])
    bGoRightB.append(barB[-1].faceleft == startFace)
    lamB.append(barsint[1][1])
    geopaths.append([startPt])
    curvs.append([0,0])
    faces.append([startFace])

finished = 0
for j in range(5000):
#def increment_path:
    for i in range(len(thetas)):
        if barF[i] != None:
            if bGoRightF[i]:
                faces[i].append(barF[i].faceright)
            else:
                faces[i].append(barF[i].faceleft)
            #print('face: ',faces[-1])
            c, barF[i], lamF[i], bGoRightF[i] = GeoCrossBar(geopaths[i][-1], barF[i], lamF[i], bGoRightF[i])
            geopaths[i].append(c)
            if faces[i][-1] is not None:
                CClast = geopaths[i][-1]-geopaths[i][-2]
                curvature = P3.Dot(tbm.faces[faces[i][-1]].normal,CClast)/P3.Len(tbm.faces[faces[i][-1]].normal)
                curvs[i].append(curvs[i][-1]*0.75 + curvature*0.25)

        if barB[i] != None:
            if bGoRightB[i]:
                faces[i].insert(0,barB[i].faceright)
            else:
                faces[i].insert(0,barB[i].faceleft)
            #print('face: ',faces[-1])
            c, barB[i], lamB[i], bGoRightB[i] = GeoCrossBar(geopaths[i][0], barB[i], lamB[i], bGoRightB[i])
            geopaths[i].insert(0,c)
            if faces[i][0] is not None:
                CClast = geopaths[i][0]-geopaths[i][1]
                curvature = P3.Dot(tbm.faces[faces[i][0]].normal,CClast)/P3.Len(tbm.faces[faces[i][0]].normal)
                curvs[i].insert(0,curvs[i][-1]*0.75 + curvature*0.25)
  #      if barF[i] ==None and barB[i] ==None:
  #          finished +=1
  #          print('Angle',i, 'edge reached')
  #          print(finished,len(thetas))
  #  if finished == len(thetas):
  #      print('All Edges reached')
   #     break
    #return geopaths, curvs

In [24]:
ps.init()
#for i in range(len(tbm.bars)):
#    pt1 = tbm.bars[i].GetNodeFore(True).p
#    pt2 = tbm.bars[i].GetNodeFore(False).p
#    nodes = np.array([(pt1.x,pt1.y,pt1.z),(pt2.x,pt2.y,pt2.z)])
#    ps_net = ps.register_curve_network('bar'+str(i), nodes, 'line',radius = 0.002, enabled=False)
ps_surf = ps.register_surface_mesh("my mesh", V, F, smooth_shade=False)
#ps_cloud = ps.register_point_cloud("vertices", V)
ps_SP = ps.register_point_cloud("Start", P3list2array([startPt]))
ps_SP.add_vector_quantity("ref", np.array([ref90]))
for i in range(len(geopaths)):
    ps_geopath = ps.register_curve_network(str(i), P3list2array(geopaths[i]), 'line')
    ps_geopath.add_scalar_quantity("curvature", np.array(curvs[i]), defined_on='edges', enabled=True,  cmap='coolwarm',vminmax=(-0.1, 0.1))
ps_surf.add_vector_quantity("normals", n, defined_on='faces', color=(1, 0.5, 0.5))
ps.show()

In [ ]:
ps.init()

ps_surf = ps.register_surface_mesh("my mesh", V, F, smooth_shade=False)
ps_SP = ps.register_point_cloud("Start", P3list2array([startPt]))
ps_SP.add_vector_quantity("ref", np.array([ref90]))
for i in range(len(geopaths)):
    ps_geopath = ps.register_curve_network(str(i), P3list2array(geopaths[i]), 'line')
    ps_geopath.add_scalar_quantity("curvature", np.array(curvs[i]), defined_on='edges', enabled=True,  cmap='coolwarm',vminmax=(-0.1, 0.1))
ps.show()

In [8]:
geopaths_sub = []
ps.init()

def callback():
    # Executed every frame
    # Do computation here, define custom UIs, etc.
    ps_geopath = ps.register_curve_network(str(i), P3list2array(geopaths[i]), 'line')
    ps_geopath.add_scalar_quantity("curvature", np.array(curvs[i]), defined_on='edges', enabled=True,  cmap='coolwarm',vminmax=(-0.1, 0.1))
    i+=1

ps_surf = ps.register_surface_mesh("my mesh", V, F, smooth_shade=False)
ps_SP = ps.register_point_cloud("Start", P3list2array([startPt]))
ps_SP.add_vector_quantity("ref", np.array([ref90]))

for i in range(len(geopaths_sub)):
    ps_geopath = ps.register_curve_network(str(i), P3list2array(geopaths[i]), 'line')
    ps_geopath.add_scalar_quantity("curvature", np.array(curvs[i]), defined_on='edges', enabled=True,  cmap='coolwarm',vminmax=(-0.1, 0.1))        

ps.set_user_callback(callback)

ps.show()

ps.clear_user_callback()

[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 4.6 (Core Profile) Mesa 21.2.6


UnboundLocalError: local variable 'i' referenced before assignment

In [41]:
seedVec = (0.1,0.2,0)
geopath = []
faces =[]
lam=0.6
bar = tbm.bars[1]
bGoRight = False
ptD = pointOnEdge(1,1-lam)

c = P3(ptD.x-seedVec[0],ptD.y-seedVec[1],ptD.z-seedVec[2])
curv = [1]


geopath.append(c)
for i in range(5000):
    
    if bar != None:
        if bGoRight:
            faces.append(bar.faceright)
        else:
            faces.append(bar.faceleft)
        #print('face: ',faces[-1])
        c, bar, lam, bGoRight = GeoCrossBar(c, bar, lam, bGoRight,False)
        geopath.append(c)
        if faces[-1] is not None:
            CClast = geopath[-1]-geopath[-2]
            curvature = P3.Dot(tbm.faces[faces[-1]].normal,CClast)/P3.Len(tbm.faces[faces[-1]].normal)
            curv.append(curv[-1]*0.75 + curvature*0.25)
            #curv.append( curvature)

    else:
        print('Edge reached')
        break
#print(geopath)

Edge reached
